# Tests



In [ ]:
from openmm import app
from openmm import *
from openmm import unit as u
import molsysmt as msm
from sys import stdout

In [ ]:
pdb = app.PDBFile('V5.pdb')

In [ ]:
modeller = app.Modeller(pdb.topology, pdb.positions)

In [ ]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [ ]:
modeller.addSolvent(forcefield, model='tip3p', padding=1.4*u.nanometers, boxShape='dodecahedron')

In [ ]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, file="V5_solvated.pdb")

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.2*u.nanometer, constraints=app.HBonds)

In [ ]:
friction = 1 / u.picosecond
timestep = 2 * u.femtoseconds
temperature = 500 * u.kelvin
integrator = LangevinIntegrator(temperature, friction, timestep)

In [ ]:
platform = Platform.getPlatformByName("CUDA")  # Usa "CPU" si no tienes GPU

In [ ]:
simulation = app.Simulation(modeller.topology, system, integrator, platform)

In [ ]:
simulation.context.setPositions(modeller.positions)

In [ ]:
simulation.minimizeEnergy()

In [ ]:
reporter1 = app.DCDReporter('traj.dcd', 2000)
reporter2 = app.StateDataReporter(stdout, 10000, step=True, potentialEnergy=True, temperature=True)

In [ ]:
simulation.reporters = [reporter1, reporter2]

In [ ]:
simulation.step(50000)

In [ ]:
molsys = msm.convert(['V5_solvated.pdb','traj.dcd'])

In [ ]:
msm.info(molsys)

In [ ]:
msm.view(molsys, standard=True)